### Purpose of this is to try and coincide our pipeline with the permit excel sheet provided

In [1]:
# Generic
import pyodbc
import os
import sys
import pandas as pd

os.chdir('../')
# Modules
import sql
import main
from store import Store
from defaults import *
sql.set_up()

In [2]:
active = sql.query("SELECT * from v_active")

In [3]:
active.head(1)

,status,combined_address,zipcode
0,Active,6101 FEGENBUSH LN,40228


In [9]:
# https://stackoverflow.com/questions/29876184/groupby-results-to-dictionary-of-lists
# Group by zipcodes
groups = active.groupby(by = 'zipcode')['combined_address'].apply(list).to_dict()

In [11]:
# Get defaults for store_types
from defaults import store_types, zipcodes
# Pick location to try to find with pipeline
p_zipcodes =  list(groups.keys())
p_zipcodes.remove('     ') # two addresses don't have zipcodes
idx = 0
target_zip = p_zipcodes[idx]
print(f"Looking for {groups[target_zip]} in {target_zip}")


Looking for ['5929 DR TIMBER RIDGE', '5949 DR TIMBER RIDGE', '6501 LN FOREST COVE', '5900 DR TIMBER RIDGE'] in 40059


In [13]:
# Find stores
stores = []
zipcodes = [target_zip] # commnet for all zipcodes
query_types = store_types[:1] # comment  for all store types
stores = main.get_stores(zipcodes, query_types, 'streetview_images')


In [17]:
query_types

['tobacco']

In [14]:
print(stores[0].place)

{'business_status': 'OPERATIONAL', 'formatted_address': '13206 W U.S. Hwy 42 suite 101, Prospect, KY 40059, United States', 'geometry': {'location': {'lat': 38.3644305, 'lng': -85.60185729999999}, 'viewport': {'northeast': {'lat': 38.36586927989272, 'lng': -85.60035682010728}, 'southwest': {'lat': 38.36316962010727, 'lng': -85.60305647989271}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/shopping-71.png', 'icon_background_color': '#4B96F3', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/shopping_pinlet', 'name': "TJ's Smoke Outlet", 'opening_hours': {'open_now': False}, 'photos': [{'height': 4032, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/109474097227940627758">A Google User</a>'], 'photo_reference': 'AcJnMuFgOcj2MglnWM5cS3hBDKSdInWBnJOuX6USEEaGu8FPry-Ya4txqap_sFeuYHFFnmdYqhU2dOMd8srhPIJXj2_UDfq3iM7E1CogNEExcC8ysYjCF63sS7CvjcWpkgJ0etzNmKYdbpvAgDMEv013pNdnU1BGO7VKIqs17UuEZmMsagKQ', 'width': 3024}], 'place_

In [15]:
print(f"Number of stores pulled through pipeline for zip {target_zip}: {len(stores)}")
found = 0
places = [s.place for s in stores]
for place in places:
    place_add = place['formatted_address'].lower()
    #print(place['formatted_address'].lower())
    #https://stackoverflow.com/questions/3271478/check-list-of-words-in-another-string
    if any(add.lower() in place_add for add in groups[target_zip]):
        #print(f"Found location of store {place} at stores[index] of {places.index(place)}")
        found += 1
print(f"Found {found} permits of {len(groups[target_zip])} in zipcode {target_zip}")
        


Number of stores pulled through pipeline for zip 40059: 1
Found 0 permits of 4 in zipcode 40059


# Full Variant

In [29]:
print(f"Our txt file has {len(zipcodes)} versus {len(groups.keys())} zipcodes on file")
found_places = {}
iterations = 10
for target_zip in p_zipcodes: #list(groups.keys())[1:] -L> has empty first so if you do anything you need to list[1:]
    #print(f"Looking for {groups[target_zip]} in {target_zip}")
    stores = main.get_stores(target_zip, store_types, 'streetview_images')
    
    print(f"Number of stores pulled through pipeline for zip {target_zip}: {len(stores)}")
    found = 0
    places = [s.place for s in stores]
    places_address = set([p['formatted_address'].lower() for p in places])
    for place_add in places_address:
        #print(place['formatted_address'].lower())
        #https://stackoverflow.com/questions/3271478/check-list-of-words-in-another-string
        if any(add.lower() in place_add for add in groups[target_zip]):
            #print(f"Found location of store {place} at stores[index] of {places.index(place)}")
            found += 1
    print(f"Found {found} permits of {len(groups[target_zip])} in zipcode {target_zip}")
    
    iterations -= 1
    if iterations == 0: break

Our txt file has 1 versus 34 zipcodes on file
Error Deleting: stat: path should be string, bytes, os.PathLike or integer, not NoneType
Error Deleting: stat: path should be string, bytes, os.PathLike or integer, not NoneType
Error Deleting: stat: path should be string, bytes, os.PathLike or integer, not NoneType
Error Deleting: stat: path should be string, bytes, os.PathLike or integer, not NoneType
Error Deleting: stat: path should be string, bytes, os.PathLike or integer, not NoneType
Error Deleting: stat: path should be string, bytes, os.PathLike or integer, not NoneType
Error Deleting: stat: path should be string, bytes, os.PathLike or integer, not NoneType
Error Deleting: stat: path should be string, bytes, os.PathLike or integer, not NoneType
Error Deleting: stat: path should be string, bytes, os.PathLike or integer, not NoneType
Error Deleting: stat: path should be string, bytes, os.PathLike or integer, not NoneType
Error Deleting: stat: path should be string, bytes, os.PathLike 

In [153]:
places_address = [p['formatted_address'] for p in places]

In [25]:
groups[target_zip]

['937 LN PHILLIPS',
 '4309 DR CRITTENDEN',
 '1016 LN PHILLIPS',
 '4300 DR CRITTENDEN',
 '2704 DR CRITTENDEN']

In [26]:
places_address

{'1102 algonquin pkwy, louisville, ky 40208, united states',
 '2204 heather ln, louisville, ky 40218, united states',
 '2221 crittenden dr, louisville, ky 40217, united states',
 '2911 preston hwy a, louisville, ky 40217, united states',
 '3022 s 3rd st, louisville, ky 40208, united states',
 '3137 s 2nd st, louisville, ky 40208, united states',
 '3210 preston hwy, louisville, ky 40213, united states',
 '3337 newburg rd, louisville, ky 40218, united states',
 '346 e lee st, louisville, ky 40208, united states',
 '3945 s 3rd st, louisville, ky 40214, united states',
 '4017 preston hwy, louisville, ky 40213, united states',
 '427 warnock st, louisville, ky 40217, united states',
 '4537 poplar level rd, louisville, ky 40213, united states',
 '5013 preston hwy, louisville, ky 40213, united states',
 '5330 s 3rd st #101, louisville, ky 40214, united states',
 '5400 preston hwy b1, louisville, ky 40213, united states',
 '5401 indian oaks cir, louisville, ky 40219, united states',
 '5520 fern